In [1]:
# GEREKLI KUTUPHANELERIN EKLENMESI

import pandas as pd
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import BertTokenizer, TFBertForSequenceClassification
import tensorflow as tf
import torch

In [3]:
# CSV DOSYASININ OKUNMASI

dframe = pd.read_csv("/content/vodafone_datasett.csv", encoding='utf_8', on_bad_lines = 'skip', sep = ';')
dframe

,Title,Explanation,Target,Link
0,NaN,Vodafone'un hazırlamış olduğu FreeZone ve fatu...,0,https://www.sikayetvar.com/vodafone/vodafone-f...
1,NaN,"Vodafone 50 GB herkese verdi, bana vermedi. Be...",0,https://www.sikayetvar.com/vodafone/vodafone-5...
2,NaN,Vodafone müşteri hizmetleri numarası 542 yi ar...,0,https://www.sikayetvar.com/vodafone/vodafone-m...
3,NaN,Merhabalar Vodafone 50 GB hediye internet kamp...,0,https://www.sikayetvar.com/vodafone/vodafone-5...
4,NaN,Merhabalar Vodafone 50 GB hediye internet kamp...,0,https://www.sikayetvar.com/vodafone/vodafone-5...
...,...,...,...,...
1770,NaN,Vodafone’un mobil uygulaması genellikle kullan...,2,NaN
1771,NaN,"Vodafone’un kapsama alanı çoğu yerde yeterli, ...",2,NaN
1772,NaN,Vodafone’un sunduğu kampanya ve fırsatlar gene...,2,NaN
1773,NaN,"Vodafone’un internet hızları genellikle iyi, a...",2,NaN


In [4]:
# TITLE VE LINK SUTUNLARININ TABLODAN CIKARILMASI

#dframe = dframe.iloc[300:]
dframe = dframe.drop(columns="Title",axis = 1)
dframe = dframe.drop(columns="Link",axis=1)
#dframe = dframe.sample(frac = 1)
dframe

,Explanation,Target
0,Vodafone'un hazırlamış olduğu FreeZone ve fatu...,0
1,"Vodafone 50 GB herkese verdi, bana vermedi. Be...",0
2,Vodafone müşteri hizmetleri numarası 542 yi ar...,0
3,Merhabalar Vodafone 50 GB hediye internet kamp...,0
4,Merhabalar Vodafone 50 GB hediye internet kamp...,0
...,...,...
1770,Vodafone’un mobil uygulaması genellikle kullan...,2
1771,"Vodafone’un kapsama alanı çoğu yerde yeterli, ...",2
1772,Vodafone’un sunduğu kampanya ve fırsatlar gene...,2
1773,"Vodafone’un internet hızları genellikle iyi, a...",2


In [5]:
# TABLODAKI DUYGULARIN SAYISI

print("0: Olumsuz", "1: Olumlu", "2: Nötr\n")
dframe["Target"].value_counts()

0: Olumsuz 1: Olumlu 2: Nötr



,count
Target,
0,1166
1,400
2,209


In [ ]:
# METINDEN DOLGU SOZCUKLERININ CIKARILMASI


In [7]:
# METINDEN NOKTALAMA ISARETLERINI, RAKAMLARI CIKARMA

def clean_text(text):
    text = " ".join(str(text).split())
    text = text.lower()
    text = text.replace("\\n", " ")
    text = re.sub("[0-9]+", "", text)
    text = re.sub("%|(|)|-", "", text)
    text = text.translate(str.maketrans("", "", string.punctuation))
    #text = [t for t in text.split() if t not in stop_words]
    return "".join(text)

dframe["Clean"] = dframe.apply(lambda row: clean_text(row["Explanation"]), axis=1)
dframe

,Explanation,Target,Clean
0,Vodafone'un hazırlamış olduğu FreeZone ve fatu...,0,vodafoneun hazırlamış olduğu freezone ve fatur...
1,"Vodafone 50 GB herkese verdi, bana vermedi. Be...",0,vodafone gb herkese verdi bana vermedi ben de...
2,Vodafone müşteri hizmetleri numarası 542 yi ar...,0,vodafone müşteri hizmetleri numarası yi aradı...
3,Merhabalar Vodafone 50 GB hediye internet kamp...,0,merhabalar vodafone gb hediye internet kampan...
4,Merhabalar Vodafone 50 GB hediye internet kamp...,0,merhabalar vodafone gb hediye internet kampan...
...,...,...,...
1770,Vodafone’un mobil uygulaması genellikle kullan...,2,vodafone’un mobil uygulaması genellikle kullan...
1771,"Vodafone’un kapsama alanı çoğu yerde yeterli, ...",2,vodafone’un kapsama alanı çoğu yerde yeterli a...
1772,Vodafone’un sunduğu kampanya ve fırsatlar gene...,2,vodafone’un sunduğu kampanya ve fırsatlar gene...
1773,"Vodafone’un internet hızları genellikle iyi, a...",2,vodafone’un internet hızları genellikle iyi an...


In [8]:
# SATIRLARI NUMPY DIZISI YAPMA

X = dframe["Clean"]
y = dframe["Target"]

In [9]:
# TEST VE EGITIM VERILERININ AYRILMASI

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=23)

In [10]:
# BERT TOKENIZER EKLEME

tokenizer = BertTokenizer.from_pretrained('dbmdz/bert-base-turkish-cased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/251k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

In [71]:
# BERT TOKENIZER UYGULAMA

max_len= 1024

X_train_encoded = tokenizer(list(X_train), truncation=True, padding=True, max_length = max_len)

X_test_encoded = tokenizer(list(X_test), truncation=True, padding=True, max_length = max_len)


In [72]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(X_train_encoded),
    y_train
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(X_test_encoded),
    y_test
))

# Shuffle and batch the data
train_dataset = train_dataset.shuffle(len(X_test)).batch(16)
val_dataset = val_dataset.batch(16)


In [73]:
# SINIFLANDIRMA MODELINI YUKLEME

model = TFBertForSequenceClassification.from_pretrained('dbmdz/bert-base-turkish-cased', num_labels = 3)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [74]:
# MODELI COMPILE ETME

optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [75]:
# MODELI EGITME

history = model.fit(train_dataset, validation_data=val_dataset, epochs=3)

Epoch 1/3
89/89 [==============================] - 181s 2s/step - loss: 0.2069 - accuracy: 0.9141 - val_loss: 0.0135 - val_accuracy: 0.9972
Epoch 2/3
89/89 [==============================] - 138s 2s/step - loss: 0.0223 - accuracy: 0.9930 - val_loss: 0.0028 - val_accuracy: 1.0000
Epoch 3/3
89/89 [==============================] - 138s 2s/step - loss: 0.0042 - accuracy: 0.9986 - val_loss: 7.7563e-04 - val_accuracy: 1.0000


In [76]:
# EVALUATE MODEL

test_loss, test_accuracy = model.evaluate(train_dataset)
print(f'Test loss: {test_loss}, Test accuracy: {test_accuracy}')

89/89 [==============================] - 50s 531ms/step - loss: 8.0605e-04 - accuracy: 1.0000
Test loss: 0.0008060492109507322, Test accuracy: 1.0


In [77]:
# MODELI KAYDETME

path = 'path-to-save'
# Save tokenizer
tokenizer.save_pretrained(path +'/Tokenizer')

# Save model
model.save_pretrained(path +'/Model')

In [78]:
# MODELI YUKLEME

# Load tokenizer
#bert_tokenizer = BertTokenizer.from_pretrained(path +'/Tokenizer')

# Load model
#bert_model = TFBertForSequenceClassification.from_pretrained(path +'/Model')

bert_tokenizer = tokenizer
bert_model = model

In [79]:
### DENEME

pred = bert_model.predict(train_dataset)

# pred is of type TFSequenceClassifierOutput
logits = pred.logits

# Use argmax along the appropriate axis to get the predicted labels
pred_labels = tf.argmax(logits, axis=1)

# Convert the predicted labels to a NumPy array
pred_labels = pred_labels.numpy()

label = {
	0: 'Olumsuz',
	1: 'Olumlu',
  2: 'Nötr'
}

# Map the predicted labels to their corresponding strings using the label dictionary
pred_labels = [label[i] for i in pred_labels]
Actual = [label[i] for i in y_train]

print('Predicted Label :', pred_labels[:10])
print('Actual Label :', Actual[:10])


89/89 [==============================] - 50s 529ms/step
Predicted Label : ['Olumsuz', 'Olumsuz', 'Olumlu', 'Olumsuz', 'Olumsuz', 'Olumsuz', 'Olumsuz', 'Olumsuz', 'Olumlu', 'Nötr']
Actual Label : ['Olumsuz', 'Nötr', 'Olumsuz', 'Olumsuz', 'Olumsuz', 'Olumlu', 'Nötr', 'Olumlu', 'Olumlu', 'Olumsuz']


In [80]:
print("Classification Report: \n", classification_report(Actual, pred_labels))

Classification Report: 
               precision    recall  f1-score   support

        Nötr       0.10      0.10      0.10       173
      Olumlu       0.19      0.19      0.19       319
     Olumsuz       0.65      0.65      0.65       928

    accuracy                           0.48      1420
   macro avg       0.31      0.31      0.31      1420
weighted avg       0.48      0.48      0.48      1420



In [81]:
# KULLANICI GIRISI ILE TAHMIN FONKSIYONU

def Get_sentiment(Review, Tokenizer=bert_tokenizer, Model=bert_model):
	# Convert Review to a list if it's not already a list
	if not isinstance(Review, list):
		Review = [Review]

	Input_ids, Token_type_ids, Attention_mask = Tokenizer.batch_encode_plus(Review,
																			padding=True,
																			truncation=True,
																			max_length=128,
																			return_tensors='tf').values()
	prediction = Model.predict([Input_ids, Token_type_ids, Attention_mask])

	# Use argmax along the appropriate axis to get the predicted labels
	pred_labels = tf.argmax(prediction.logits, axis = 1)

	# Convert the TensorFlow tensor to a NumPy array and then to a list to get the predicted sentiment labels
	pred_labels = [label[i] for i in pred_labels.numpy().tolist()]
	return pred_labels


In [82]:
# KULLANICI GIRISI ILE TAHMIN FONKSIYONU

Review = "Vodafone nin sundugu hizmetlerden cok memnum kaldim. Diger operatorlerden daha iyi daha guzel. Vodafone ye gecmek hayatimda aldigim en guzel karardi. Akrabalarima da vodafone ye gecmeleri konusunda ısrarcı olacagim. Fiyatlari cazip ve guzel. Tesekkur ederim."
Get_sentiment(Review)

1/1 [==============================] - 4s 4s/step


['Olumlu']

In [83]:
Review = "kotu berbat."
Get_sentiment(Review)

1/1 [==============================] - 4s 4s/step


['Olumsuz']

In [84]:
Review = "Cok iyi sevdim baya bunu"
Get_sentiment(Review)

1/1 [==============================] - 0s 48ms/step


['Olumlu']

In [85]:
Review = "Vodfone yi cok seviyorum. Cok memnun kaldim. Cok tesekkur ederim. İyi ki varsiniz."
Get_sentiment(Review)

1/1 [==============================] - 0s 49ms/step


['Olumlu']

In [86]:
Review = "1 haftadır arama yapamıyorum. Faturalarımı ödememe ragmen sebekeye baglanılamıyor hatas alıyorum. Bu sorunu lutfen en erken vakitte cozun. Ticari is yapiyorum. Cok zarardayim."
Get_sentiment(Review)

1/1 [==============================] - 0s 50ms/step


['Olumsuz']

In [87]:
Review = "Fiyatlar cok ucuz. Cok guzel cekiyor. Tavsiye ediyorum."
Get_sentiment(Review)

1/1 [==============================] - 0s 47ms/step


['Olumlu']

In [88]:
Review = "vodafone, müşteri hizmetleriyle ilgili genel olarak iyi bir izlenim bırakıyor, fakat bazen temsilciye ulaşmak zor olabiliyor."
Get_sentiment(Review)

1/1 [==============================] - 0s 45ms/step


['Nötr']

In [89]:
Review ="İyi günler,Çark hatası alıyorum ve her seferinde Hay aksi hatasını alıyorum. 1 aydır aynı şekilde hata vermesi çok can sıkıcı. Uygulamayı güncelledim, olmadı. Silip aynı şekilde tekrar yükledim, yine olmadı. Müşteri temsilcisi ile görüşmek istedim, bir türlü bağlanamadım. Lütfen sorunu düzeltebilir misiniz?"
Get_sentiment(Review)

1/1 [==============================] - 0s 49ms/step


['Olumsuz']

In [90]:
Review = "Her türlü cihazla uyumlu SIM kartları sayesinde teknolojiyle uyumlu bir iletişim deneyimi yaşıyorum her cihazda rahatlıkla kullanabiliyorum."
Get_sentiment(Review)

1/1 [==============================] - 0s 54ms/step


['Olumlu']